# 1. Basics of TensorFlow

In [1]:
import tensorflow as tf

In [2]:
hello_constant = tf.constant("Hello TF")

with tf.Session() as sess:
    output = sess.run(hello_constant)
    print (output)

b'Hello TF'


In [3]:
# TODO: z = x/y - 1
# Method 1
x = tf.constant(10.0)
y = tf.constant(2.0)
z = tf.subtract( tf.div(x,y), tf.cast(tf.constant(1), tf.float32) )

with tf.Session() as sess:
    z = sess.run(z)
    print("method 1, z: {:.6f}".format(z))

    
# Method 2
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = tf.placeholder(tf.float32)

with tf.Session() as sess:
    output = sess.run(tf.subtract((tf.div(x,y)), z), feed_dict={x: 10.0, y: 2.0, z: 1.0})
    print ("method 2, z: {:.6f}".format(output))

method 1, z: 4.000000
method 2, z: 4.000000


# 2. TensorFlow example with MNIST

In [4]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets('.', one_hot=True, reshape=False)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [6]:
train_x = mnist.train.images
valid_x = mnist.validation.images
test_x  = mnist.test.images

train_y   = mnist.train.labels.astype(np.float32)
valid_y   = mnist.validation.labels.astype(np.float32)
test_y    = mnist.test.labels.astype(np.float32)

In [7]:
# Set learning parameters
learning_rate   = 0.001
training_epochs = 20
batch_size      = 128

n_input   = 784 # img: 28*28
n_classes = 10  # 0-9 digits

In [8]:
# Set hidden layer parameters
n_hidden_layer = 256

In [9]:
# Input
x = tf.placeholder('float', [None, 28, 28, 1])
y = tf.placeholder('float', [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])

In [10]:
# Set layer weights and biases
weights = {
    'hidden_layer': tf.Variable(tf.truncated_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_layer, n_classes]))
}
biases  = {
    'hidden_layer': tf.Variable(tf.zeros(n_hidden_layer)),
    'out': tf.Variable(tf.zeros(n_classes))
}

In [11]:
# Multilayer perceptron
hidden_output = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
logits_input = tf.nn.relu(hidden_output)
logits = tf.add(tf.matmul(logits_input, weights['out']), biases['out'])

In [12]:
# Cost and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [13]:
# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
def print_epoch_stats(epoch_i, sess, last_x, last_y):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost   = sess.run(cost, feed_dict={x: last_x, y: last_y})
    valid_accuracy = sess.run(accuracy, feed_dict={x: valid_x, y: valid_y})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(epoch_i, current_cost, valid_accuracy))

In [15]:
# Run session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            train_feed_dict = {
                x: batch_x, 
                y: batch_y }
            sess.run(optimizer, train_feed_dict)
        print_epoch_stats(e, sess, batch_x, batch_y)
        
    # Calculate accuracy for test dataset
    test_accuracy = sess.run(accuracy, feed_dict={x: test_x, y: test_y})

print('Test Accuracy: {}'.format(test_accuracy))

Epoch: 0    - Cost: 19.1     Valid Accuracy: 0.393
Epoch: 1    - Cost: 21.3     Valid Accuracy: 0.553
Epoch: 2    - Cost: 11.0     Valid Accuracy: 0.629
Epoch: 3    - Cost: 17.4     Valid Accuracy: 0.672
Epoch: 4    - Cost: 11.3     Valid Accuracy: 0.7  
Epoch: 5    - Cost: 7.84     Valid Accuracy: 0.72 
Epoch: 6    - Cost: 8.14     Valid Accuracy: 0.736
Epoch: 7    - Cost: 8.64     Valid Accuracy: 0.752
Epoch: 8    - Cost: 6.44     Valid Accuracy: 0.762
Epoch: 9    - Cost: 6.97     Valid Accuracy: 0.772
Epoch: 10   - Cost: 6.25     Valid Accuracy: 0.782
Epoch: 11   - Cost: 5.66     Valid Accuracy: 0.787
Epoch: 12   - Cost: 6.62     Valid Accuracy: 0.794
Epoch: 13   - Cost: 5.5      Valid Accuracy: 0.8  
Epoch: 14   - Cost: 6.45     Valid Accuracy: 0.806
Epoch: 15   - Cost: 7.03     Valid Accuracy: 0.811
Epoch: 16   - Cost: 3.25     Valid Accuracy: 0.816
Epoch: 17   - Cost: 6.63     Valid Accuracy: 0.82 
Epoch: 18   - Cost: 6.83     Valid Accuracy: 0.824
Epoch: 19   - Cost: 3.19     Va